In [13]:
import pandas as pd
import numpy as np
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('../data/data/u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head(3)

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1


In [14]:
movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head(3)

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333

In [18]:
sizes = movieProperties['rating']['size'].values
scores = movieProperties['rating']['mean'].values
movieProperties["popularity"] = (sizes - sizes.mean())/sizes.max()
movieProperties["score"] = (scores - scores.mean())/scores.max() 
movieProperties.head(3)

rating           popularity     score
           size      mean                     
movie_id                                      
1           452  3.878319   0.672166  0.160456
2           131  3.206107   0.122509  0.026014
3            90  3.033333   0.052303 -0.008541

In [24]:
movieProperties['score'].hist()

In [25]:
movieProperties['popularity'].hist()

In [55]:
%%time
movie_id = []
movie_gen = []
movie_name = []
with open('../data/data/u.item', 'r') as f:
    for line in f:
        contents = line.rstrip('\n').split('|')
        movie_id.append(int(contents[0]))
        movie_name.append(contents[1])
        movie_gen.append(np.array([int(x) for x in contents[5:25]]))
mgen = pd.DataFrame({
    'movie_id': movie_id,
    'movie_name': movie_name
})
print np.array(movie_gen).shape
for i in range(len(movie_gen[0])):
    mgen['gen'+str(i)] = np.array(movie_gen)[:, i]
mgen.set_index('movie_id', inplace=True)
print(len(mgen))
print mgen.head(3)

(1682, 19)
1682
                 movie_name  gen0  gen1  gen2  gen3  gen4  gen5  gen6  gen7  \
movie_id                                                                      
1          Toy Story (1995)     0     0     0     1     1     1     0     0   
2          GoldenEye (1995)     0     1     1     0     0     0     0     0   
3         Four Rooms (1995)     0     0     0     0     0     0     0     0   

          gen8  gen9  gen10  gen11  gen12  gen13  gen14  gen15  gen16  gen17  \
movie_id                                                                       
1            0     0      0      0      0      0      0      0      0      0   
2            0     0      0      0      0      0      0      0      1      0   
3            0     0      0      0      0      0      0      0      1      0   

          gen18  
movie_id         
1             0  
2             0  
3             0  
CPU times: user 153 ms, sys: 14.3 ms, total: 167 ms
Wall time: 158 ms


In [56]:
info = movieProperties.join(mgen, how='outer')
info.columns = ['A', 'B', 'C', 'D', 'name'] + ['gen'+str(i) for i in range(19)]
print info.columns
info.head(3)

Index([u'A', u'B', u'C', u'D', u'name', u'gen0', u'gen1', u'gen2', u'gen3',
       u'gen4', u'gen5', u'gen6', u'gen7', u'gen8', u'gen9', u'gen10',
       u'gen11', u'gen12', u'gen13', u'gen14', u'gen15', u'gen16', u'gen17',
       u'gen18'],
      dtype='object')


,A,B,C,D,name,gen0,gen1,gen2,gen3,gen4,...,gen9,gen10,gen11,gen12,gen13,gen14,gen15,gen16,gen17,gen18
movie_id,,,,,,,,,,,,,,,,,,,,,
1,452,3.878319,0.672166,0.160456,Toy Story (1995),0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,131,3.206107,0.122509,0.026014,GoldenEye (1995),0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,90,3.033333,0.052303,-0.008541,Four Rooms (1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [57]:
metrics = ['name', 'C', 'D'] + ['gen'+str(i) for i in range(19)]
movie_info = info[metrics].values

In [59]:
from scipy import spatial

def computeDist(a, b):
    popu_dis = abs(a[1] - b[1])
    rating_dis = abs(a[2] - b[2])
    genr_dis = spatial.distance.cosine(a[3:], b[3:])
    return popu_dis + rating_dis + genr_dis

print "distance between ", movie_info[0][0], " and ", movie_info[4][0], " is ", computeDist(movie_info[0], movie_info[4])

distance between  Toy Story (1995)  and  Copycat (1995)  is  1.7419109293


##### compute K-nearest neightbors

In [72]:
import operator

def getNeighbors(idx, K, k=10):
    dists = []
    for j in range(len(K)):
        if j == idx:
            continue
        dist = computeDist(K[idx], K[j])
        dists.append((K[j], dist))
    dists.sort(key=lambda x: x[1])
    return dists[: k]

In [68]:
## select movies:
idx = 0
print "The selected movie is: ", movie_info[idx][0]

The selected movie is:  Toy Story (1995)


In [71]:
neighbors = getNeighbors(idx, movie_info)

In [73]:
## Out put the K-neightbor results
for mov, dis in neighbors:
    print "Move name: ", mov[0], " and its distance with the target movies is: ", dis

Move name:  Aladdin (1992)  and its distance with the target movies is:  0.546053838144
Move name:  Willy Wonka and the Chocolate Factory (1971)  and its distance with the target movies is:  0.598370106707
Move name:  Liar Liar (1997)  and its distance with the target movies is:  0.623480090754
Move name:  Full Monty, The (1997)  and its distance with the target movies is:  0.666971880489
Move name:  Monty Python and the Holy Grail (1974)  and its distance with the target movies is:  0.693153865565
Move name:  Lion King, The (1994)  and its distance with the target movies is:  0.749893687756
Move name:  Babe (1995)  and its distance with the target movies is:  0.75572897725
Move name:  Raising Arizona (1987)  and its distance with the target movies is:  0.758929885981
Move name:  Back to the Future (1985)  and its distance with the target movies is:  0.77521581774
Move name:  Beauty and the Beast (1991)  and its distance with the target movies is:  0.778663400344


In [64]:
A.sort(key=lambda x: -x)

In [65]:
A

[3, 2, 1, 1]